In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

Define paths

In [ ]:
DATASET_DIR = 'dataset'

data_1_filename = 'transactions_1.csv'
data_2_filename = 'transactions_2.csv'

data_1_filepath = os.path.join('..', DATASET_DIR, data_1_filename)
data_2_filepath = os.path.join('..', DATASET_DIR, data_2_filename)

Load data

In [ ]:
df_data_1 = pd.read_csv(data_1_filepath, index_col=0)
df_data_2 = pd.read_csv(data_2_filepath, index_col=0)

# General data overview

## Data separately

In [ ]:
df_data_1.head()

In [ ]:
df_data_1.shape

In [ ]:
df_data_2.head()

In [ ]:
df_data_2.shape

In [ ]:
df_data_1["date"] = pd.to_datetime(df_data_1["date"])
df_data_2["date"] = pd.to_datetime(df_data_2["date"])

In [ ]:
df_data_1['date'].max(), df_data_2['date'].min()

In [ ]:
plt.hist(
    [df_data_1['date'], df_data_2['date']],
    bins=20,
    stacked=True,
    label=[data_1_filename, data_2_filename],
    color=['blue', 'orange']
)
plt.legend()
plt.xticks(rotation=90)

In [ ]:
df_data_1["customer_id"].value_counts()

In [ ]:
df_data_2["customer_id"].value_counts()

## Data merged

In [ ]:
df_data = pd.concat([df_data_1, df_data_2])

In [ ]:
df_data['product_id'].unique()

In [ ]:
# There are two strange: 'Undefined' and 'Not a make'

In [ ]:
df_data.describe(include='all')

In [ ]:
df_data.duplicated().sum() # I assume that duplicates are not a mistake, but that a given customer bought several cars

In [ ]:
df_data['date'].min(), df_data['date'].max()

In [ ]:
df_data['customer_id'].unique()

In [ ]:
df_data['customer_id'].unique().min(), df_data['customer_id'].unique().max()

In [ ]:
df_data['year'] = df_data['date'].dt.year
df_data['month'] = df_data['date'].dt.month
df_data['day_of_week'] = df_data['date'].dt.dayofweek

Distribution of number of transactions per customer

In [ ]:
# In[4]:
transactions_per_customer = df_data.groupby('customer_id')['product_id'].count().reset_index()
transactions_per_customer.columns = ['customer_id', 'transaction_count']

transactions_per_customer.sort_values('transaction_count', ascending=False, inplace=True)

top_n = 20
sns.barplot(
    data=transactions_per_customer.head(top_n),
    x='customer_id', y='transaction_count',
    order=transactions_per_customer.head(top_n)['customer_id']
)
plt.xticks(rotation=45)
plt.show()

transactions per product

In [ ]:
# In[5]:
transactions_per_product = df_data.groupby('product_id')['customer_id'].count().reset_index()
transactions_per_product.columns = ['product_id', 'transaction_count']

transactions_per_product.sort_values('transaction_count', ascending=False, inplace=True)

top_n = 10
plt.figure(figsize=(10, 6))
sns.barplot(
    data=transactions_per_product.head(top_n),
    y='product_id', x='transaction_count',
    order=transactions_per_product.head(top_n)['product_id']
)
plt.show()

transactions per month

In [ ]:
monthly_transactions = df_data.groupby([df_data['date'].dt.year, df_data['date'].dt.month])['customer_id'].count().reset_index()
monthly_transactions.columns = ['year', 'month', 'transaction_count']

monthly_transactions['year_month'] = monthly_transactions['year'].astype(str) + '-' + monthly_transactions['month'].astype(str)

plt.figure(figsize=(12, 6))
sns.lineplot(data=monthly_transactions, x='year_month', y='transaction_count', marker='o')
plt.xticks(rotation=45)
plt.show()


Transactions of a specific product in given year

In [ ]:
CHOSEN_YEAR = 2018

In [ ]:
df_data_in_chosen_year = df_data[df_data['year'] == CHOSEN_YEAR]

In [ ]:
df_data_in_chosen_year.value_counts()

In [ ]:
for product_of_interest in df_data['product_id'].unique():

    product_in_chosen_year = df_data_in_chosen_year[df_data_in_chosen_year['product_id'] == product_of_interest]
    monthly_product_in_chosen_year = product_in_chosen_year.groupby(product_in_chosen_year['date'].dt.month)['customer_id'].count().reset_index()
    monthly_product_in_chosen_year.columns = ['month', 'transaction_count']

    plt.figure(figsize=(8, 5))
    sns.barplot(data=monthly_product_in_chosen_year, x='month', y='transaction_count')
    plt.title(product_of_interest)
    plt.show()


Seasonality analysis and time of week

In [ ]:
# In[13]:
df_data['hour'] = df_data['date'].dt.hour

dow_hour = df_data.groupby(['day_of_week', 'hour'])['customer_id'].count().reset_index()
dow_hour_pivot = dow_hour.pivot(index='day_of_week', columns='hour', values='customer_id')

sns.heatmap(dow_hour_pivot, cmap="YlGnBu")
plt.title("Rozkład liczby transakcji w zależności od dnia tygodnia i godziny")
plt.xlabel("Godzina")
plt.ylabel("Dzień tygodnia (0=Poniedziałek, 6=Niedziela)")
plt.show()


# Solve tasks

## task 1

task 1: `Create an ordered (descending) plot that shows the total number of transactions per customer from the most active customer to the least active one.`

In [ ]:
df_transactions_per_customer = df_data.groupby("customer_id").size().reset_index(name="transaction_count")
df_transactions_per_customer.sort_values(by="transaction_count", ascending=False, inplace=True)

In [ ]:
plt.bar(df_transactions_per_customer["customer_id"].astype(str), df_transactions_per_customer["transaction_count"])

## task 2

task 2: `Given any product ID, create a plot to show its transaction frequency per month for the year 2018.`

In [ ]:
df_data_2018 = df_data[df_data["date"].dt.year == 2018]

In [ ]:
df_data['product_id'].unique().tolist()

In [ ]:
product_id = "Opel"

In [ ]:
product_data = df_data_2018[df_data_2018["product_id"] == product_id]

product_data["year_month"] = product_data["date"].dt.to_period("M")
monthly_transactions = product_data.groupby("year_month").size().reset_index(name="transaction_count")

plt.plot(monthly_transactions["year_month"].astype(str), monthly_transactions["transaction_count"], marker="o", color="skyblue")

## task 4

task 4: `At any time, what are the top 5 products that drove the highest sales over the last six months? Do you see a seasonality effect in this data set?`

In [ ]:
chosen_date = '2019-01-01'

In [ ]:
chosen_date = pd.Timestamp(chosen_date).tz_localize('UTC')

six_months_earlier = chosen_date - pd.DateOffset(months=6)
df_data_last_six_months = df_data[(df_data["date"] >= six_months_earlier) & (df_data["date"] <= chosen_date)]

product_sales = df_data_last_six_months.groupby("product_id").size().reset_index(name="transaction_count")
top_products = product_sales.sort_values(by="transaction_count", ascending=False).head(5)

In [ ]:
plt.bar(top_products["product_id"], top_products["transaction_count"], color="skyblue")